In [1]:
from bs4 import BeautifulSoup
import os
import requests
import urllib
import shutil
import re
import pandas as pd
import numpy as np

In [2]:
records = pd.read_csv('records.csv', index_col=0)

In [3]:
for i in range(1, 5):
    records['color{}'.format(i)] = np.zeros(360)


In [4]:
records

,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0.0,0.0,0.0,0.0
1,Air Force,243,278,0.466411,668.757677,USAF Academy,CO,1102,0.0,0.0,0.0,0.0
2,Akron,364,198,0.647687,793.686137,Akron,OH,1103,0.0,0.0,0.0,0.0
3,Alabama,324,238,0.576512,801.734237,Tuscaloosa,AL,1104,0.0,0.0,0.0,0.0
4,Alabama A&M,151,340,0.307536,485.358751,Normal,AL,1105,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
355,Wright St,309,236,0.566972,737.454923,Dayton,OH,1460,0.0,0.0,0.0,0.0
356,Wyoming,252,285,0.469274,665.159126,Laramie,WY,1461,0.0,0.0,0.0,0.0
357,Xavier,401,175,0.696181,834.066729,Cincinnati,OH,1462,0.0,0.0,0.0,0.0
358,Yale,270,228,0.542169,778.892366,New Haven,CT,1463,0.0,0.0,0.0,0.0


In [5]:
url = "https://dynasties.operationsports.com/team-colors.php?sport=ncaa"
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
lst = soup.select('td')

In [6]:
teamColorsDict= {}
for item in lst[1:]:
    team = item.select('h3')[0].text
    color_boxes = item.select('.team-color-box')
    colors = [box['style'].strip(';').split(':')[1] for box in color_boxes][::-1]
    teamColorsDict[team] = colors

In [7]:
ct = 0
for row in records.itertuples():
    team_name = row.team_name
    if team_name.startswith('E '):
        team_name2 = team_name.replace('E ','Eastern ' )
    elif team_name.startswith('NC '):
        team_name2 = team_name.replace('NC ','North Carolina ' )
    elif team_name.startswith('S ') and team_name != 'S Illinois':
        team_name2 = team_name.replace('S ','South ' )
    elif team_name.startswith('CS '):
        team_name2 = team_name.replace('CS ','Cal State ' )
    elif team_name.startswith('FL '):
        team_name2 = team_name.replace('FL ','Florida ' )
    elif ('Intl') in team_name:
        team_name2 = team_name.replace('Intl','International' )
    elif team_name.startswith('G '):
        team_name2 = team_name.replace('G ','George ' )
    elif team_name.startswith('Loy Marymount'):
        team_name2 = team_name.replace('Loy Marymount','Loyola Marymount' )
    elif team_name.startswith('Miami'):
        team_name2 = team_name.replace('FL','(FL)' )
        team_name2 = team_name2.replace('OH','(OH)' )
    elif team_name.startswith('Missouri KC'):
        team_name2 = team_name.replace('Missouri ','UM' )
    elif team_name.startswith('Monmouth'):
        team_name2 = team_name.replace('NJ','' )
    elif team_name.startswith('MD E'):
        team_name2 = team_name.replace('MD E','Maryland-Eastern' )
    elif team_name.startswith('MS '):
        team_name2 = team_name.replace('MS ','Mississippi ' )
    elif team_name.startswith('MTSU'):
        team_name2 = team_name.replace('MTSU','Middle Tennessee ' )
    elif team_name.startswith('N '):
        team_name2 = team_name.replace('N ','Northern ' )
    elif ('Ole Miss') in team_name:
        team_name2 = 'Mississippi Rebels'
    elif 'S Illinois' in team_name:
        team_name2 = 'Southern Illinois'
    else:
        team_name2 = team_name
#     print(team_name, ' --- ', team_name2)
    for k, colors in teamColorsDict.items():
        if team_name2 in k:
#             print(colors)
            for i, val in enumerate(colors, start=1):
                records['color{}'.format(i)][records.team_name == team_name] = val
            
            ct += 1
            break
            
print("Number of teams colors extracted for: ", ct)

/Users/jared/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Number of teams colors extracted for:  297


In [8]:
records.head()

,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0,0,0,0
1,Air Force,243,278,0.466411,668.757677,USAF Academy,CO,1102,#0061AA,#FFFFFF,0,0
2,Akron,364,198,0.647687,793.686137,Akron,OH,1103,#000E41,#C29C41,#FFFFFF,0
3,Alabama,324,238,0.576512,801.734237,Tuscaloosa,AL,1104,#A80532,#FFFFFF,#CACCCE,0
4,Alabama A&M,151,340,0.307536,485.358751,Normal,AL,1105,#750000,#FFFFFF,0,0


In [9]:
missing_colors = records[records.color1 == 0]
print('Missing colors for ', missing_colors.shape[0], 'teams')
display(missing_colors)

Missing colors for  63 teams


,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0,0,0,0
6,Albany NY,295,264,0.527728,594.312954,Albany,NY,1107,0,0,0,0
12,Ark Little Rock,262,265,0.497154,704.600947,Little Rock,AR,1114,0,0,0,0
13,Ark Pine Bluff,172,367,0.319109,456.117652,Pine Bluff,AR,1115,0,0,0,0
19,BYU,397,167,0.703901,848.059116,Provo,UT,1140,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
339,W Michigan,286,265,0.519056,649.613823,Kalamazoo,MI,1444,0,0,0,0
340,W Salem St,26,81,0.242991,554.680460,Winston-Salem,NC,1445,0,0,0,0
341,WI Green Bay,288,245,0.540338,678.137077,Green Bay,WI,1453,0,0,0,0
342,WI Milwaukee,275,265,0.509259,593.566394,Milwaukee,WI,1454,0,0,0,0


In [10]:
records[records.team_name.str.contains('Intl')]

,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4
95,Florida Intl,184,333,0.355899,688.790679,Miami,FL,1198,#002D62,#C5960B,#FFFFFF,0


In [11]:
colors = pd.read_csv('team_colors.csv', index_col=0)

In [12]:
colors = colors.sort_values(by='team_name') 

In [13]:
# colors.to_csv('team_colors.csv')

### create script to use with preferences.tps

In [14]:
script = ""
for hexcode in colors.color1:
    script += "<color>" + hexcode.upper() + "</color>"


## Scraping Logos

In [15]:
records.insert(loc=12, column='logo_url', value='')

In [16]:
logo_urls = {}
team = item.select('h3')[0].text

for tag in lst[1:]:
    url = (tag['style']
              .rstrip(") no-repeat left; background-size:110px 110px;") 
              .lstrip("background:url(")
              .strip("'")
             )
    team = tag.select('h3')[0].text
    logo_urls[team] = url
    

In [17]:
ct = 0
for row in records.itertuples():
    team_name = row.team_name
    if team_name.startswith('E '):
        team_name2 = team_name.replace('E ','Eastern ' )
    elif team_name.startswith('NC '):
        team_name2 = team_name.replace('NC ','North Carolina ' )
    elif team_name.startswith('S ') and team_name != 'S Illinois':
        team_name2 = team_name.replace('S ','South ' )
    elif team_name.startswith('CS '):
        team_name2 = team_name.replace('CS ','Cal State ' )
    elif team_name.startswith('FL '):
        team_name2 = team_name.replace('FL ','Florida ' )
    elif ('Intl') in team_name:
        team_name2 = team_name.replace('Intl','International' )
    elif team_name.startswith('G '):
        team_name2 = team_name.replace('G ','George ' )
    elif team_name.startswith('Loy Marymount'):
        team_name2 = team_name.replace('Loy Marymount','Loyola Marymount' )
    elif team_name.startswith('Miami'):
        team_name2 = team_name.replace('FL','(FL)' )
        team_name2 = team_name2.replace('OH','(OH)' )
    elif team_name.startswith('Missouri KC'):
        team_name2 = team_name.replace('Missouri ','UM' )
    elif team_name.startswith('Monmouth'):
        team_name2 = team_name.replace('NJ','' )
    elif team_name.startswith('MD E'):
        team_name2 = team_name.replace('MD E','Maryland-Eastern' )
    elif team_name.startswith('MS '):
        team_name2 = team_name.replace('MS ','Mississippi ' )
    elif team_name.startswith('MTSU'):
        team_name2 = team_name.replace('MTSU','Middle Tennessee ' )
    elif team_name.startswith('N '):
        team_name2 = team_name.replace('N ','Northern ' )
    elif ('Ole Miss') in team_name:
        team_name2 = 'Mississippi Rebels'
    elif 'S Illinois' in team_name:
        team_name2 = 'Southern Illinois'
    else:
        team_name2 = team_name
#     print(team_name, ' --- ', team_name2)
    for team, logo_url in logo_urls.items():
        if (team_name2 in team) and (logo_url not in records.logo_url.values):
#             print(logo_url)
            
            records['logo_url'][records.team_name == team_name] = logo_url
            
            ct += 1
            break
            
print("Number of teams colors extracted for: ", ct)

/Users/jared/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Number of teams colors extracted for:  295


In [18]:
missing_logos = records[records.logo_url == '']
print('Missing logos for ', missing_logos.shape[0], 'teams')
display(missing_logos)

Missing logos for  65 teams


,team_name,wins,losses,w/l_ratio,most_recent_elo,team_city,team_state,team_id,color1,color2,color3,color4,logo_url
0,Abilene Chr,78,106,0.423913,687.653203,Abilene,TX,1101,0,0,0,0,
6,Albany NY,295,264,0.527728,594.312954,Albany,NY,1107,0,0,0,0,
12,Ark Little Rock,262,265,0.497154,704.600947,Little Rock,AR,1114,0,0,0,0,
13,Ark Pine Bluff,172,367,0.319109,456.117652,Pine Bluff,AR,1115,0,0,0,0,
19,BYU,397,167,0.703901,848.059116,Provo,UT,1140,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,W Michigan,286,265,0.519056,649.613823,Kalamazoo,MI,1444,0,0,0,0,
340,W Salem St,26,81,0.242991,554.680460,Winston-Salem,NC,1445,0,0,0,0,
341,WI Green Bay,288,245,0.540338,678.137077,Green Bay,WI,1453,0,0,0,0,
342,WI Milwaukee,275,265,0.509259,593.566394,Milwaukee,WI,1454,0,0,0,0,


In [19]:
logos = records[['team_name', 'team_id', 'logo_url']]

In [20]:
# logos.to_csv('logos_url.csv')

In [21]:
default_address = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOIAAADfCAMAAADcKv+WAAAAkFBMVEUAXrj///8AVbUAWrcAT7MAXLcAWLYAVLWhud1ah8gAUbQATLLj7PYAULM3dcGSr9n4+/3S3u+dttxjjstLf8Xt8vnb4/G0x+SEpNTM2e2Nqtcrbr6twuLz9/ve5/MGYbl6ndG7zOZslM1cickYZrt9n9KwxOK7zOfF0+pGfMQARbA0csCOrdhzmc8AP64AO63zaPIjAAAPWklEQVR4nOWd53qjMBaGKRLY4I5rgu24Jc4k2b3/u1skmjhqYGMb2O/PPDODy2tJp6kZZudlPPPDJpPJNlL0xzM/9fGIk901HH8Eo7PhuB6i8lzHOI+Cj3F43T2e9pGIk2m4nuPh0HMtx8aRjEzkb7Zjud5wiOeXcPpI0EchLnvHsx+1ls1wiUVQPf987C0f9FUegbic9a2hpYcrgkYvmc8egVk34uTr6KCKeCymc1zV3WlrRdyGc9+9DS/HdP15uK3zW9WIuOoj174HLxWh/KyvLetCnP659fCllN5xWtNXqwcxPCPnrv7JC9vo3KulKWtAXKw9t2a+hNL1LoMGIO6C2huQgXRQcLcfuRNx2fdrHIEi2Wh+56C8C3H3cEAK6c/vask7EBfBMwBjyGDxCsQ1cp4DSCHR+mbreiti6FjPAySyrPCpiLuR9zArKhP2fnbPQ1z7TwekkP76SYjT/ZP7aC5rv3kG4uU1TRgL+x8PR9ydX9aEsaxzVSdZEbGHXtiEsbA/eyRi33s1IJE3r+QjqyDujCc6e5UcXKWzVkD8fH0nTYVRhTigPOLFfzUYK3SpH3HuvpqqKHdeM+L2vSHDMJdzLlmnK4c4wE9Km6rItstVPUohLh9Tm7lX2CtVDyiDuHllyKYS9suErCUQr00lJIzXOhAbTEgYD/cjNpowkr4ddYiNHYeZtONRg7hsTtAmE0Yau6pGHDy/RFNd2FPXdJSI2/vmCp8l7CirrErEcwNjGpHs862I88bFpTI537chXhqWW6jk/t2CGDYqP9RJkSNLEZetIozco7TWIUOc4FYY01wYy2pWMsT2mJpUtqwMIEGcNaKaWE3euArirmUDMZZkOIoRzy0biLHwvjzix4vnLW6VJfSOIsRpK7spkTCxEiEareymRBiXQ1y3tJsSWYIiOY/YTmuaSmBVecRRa7spEf7RI4YtdPqsPC4e5xCtVjdi1IwWjFUhYpttTSwLrlwBiAv06m94v9BCiXhsXYLByw5UiO12GKn8nQLx1JKSm1p2X47YjUaE/r+A2I1GhM3IIg46YE5joZ0E8a8D5jSWcxQjboev/mb1ifWNDOK49YFNLutNiPi4HSXPF3ZEiF8tTzGKclcCxO8ONWLUjCMesTseIxYacIhvHTI2RM4bh7jvVD+NeqoBETcdcoqxhhuA+NGZyCaV8wcQ27E2o4qwVUTcdMopxvI2BcR15/pp1FMvBcSu2VOi1KbGf+w65vdjJVljjNjrmN+PZc0YxH5HKhpFJQscYsRmLm+/W16OOO3kUEzdBkWcdXIoRoNxnCF2cyimxUaK2KWSBqs4hjM6mA3nonkxQTy0aOFpNblfCeJbBwPUWDT1NzpsbRJ7QxDbu5RIJ2zHiNvOWpvI3mwp4rJzZZtcZIej0WWDGptUo7vhGxHJp4wuFt9ykTKc0Z3pb5GI14gQ3zvrMyKvcaaIbduRUUVkDa5hTjpsUCOTOokQu+z5qe83OpxKEUXplGEuO40YhTeGOe3gdEYuitjFGZtc3jVCvHbboh4iRG0Uji0oGPHxT3CP0BPQXSj2WcGbMAK+21E9W0D8ihC/NIj4ewwFFpDj716vN0uUPPJbYLRd5Mwvs3BF9Ln6jBQm6vVCOvNH30SqsDh1Zr2pnj2ziKsSiMwinUzFmM/95J9gjBh20SlUnb5DUx3NwZmFT2TXRgnEFmpKIcLV5ESfhdf4gieyOVk8fA81x7bRzS9IfQTRnEW0j8pnjxBRMxbxu+hd2OKy8ImkY+Hhj/bUHbqCEitPHzDNgE2HLEHHYnRhBgk1N1e103DeRO/CLkQSPZGETI590AEmVQdHc+gZ2zKGrz5Y65f5cu5V7xfF5zttmVXkopNK6JwsRooDBXLRlb6e5tgatmUE+6L4D0+//kYb3WRLO4DyjiN84tsmZuZQhtDc099JM17ZxbKWZF90KtZS0OhmpyzAwX0dqfLIVvQEyV7sc7lTouk6A6w6JIOILTDpDrhhh95wGSGq9xCJHAJVtgdQ9EQ0vJyfksdf0m5l6Q4EDfOWEW41ZcX2y8gfRCmxElHkEKiyFboip/LnOCP+X8Waky6PdCctM3bfUbsMs9AvhyQlVs4uil0GVVIPEY59rD6BpiBUpmHYzqcd4ky/JEGCod7E78hHdmIARE9MkVP6XhP63W3tCcRM53N1z06YjrqniKoV07j/8fd3FHbWSew38DwIgn4xNBn/kxiEzWe4+iy+WxygjoLg9Kv62kzn2x+jD1SasrxfkvXTBvCqHKPjOJKfLdm5gu0ohyj+z1n4bRd/nhdlF/+KZijuQ9GbiMY082LGYkTPqg8Qz2N2Yu4N/QyqbHjnm8hAzLgQjqxfn/644ECTrHKE1EFZoYimi/bymJ1EXkbBHgslSjWovtPmp/PNuXqCsHQySj4FuIf0w8GGPF7spjwfjgPgn+ZZvyT5i6FfiyqNCDMrh4pPCLrpdp92FeAektkG7MIP6YG/M63IHaWxApv589CLxIWGdsGGIiJM04miD9zytmaSHbeWr0qPlSxN47rpB4ztmalsOA4m8LyCfCKKpGjkE9WIiogwiXfBE4LE7zv7TDyaMeWDWZJWOifwguk/aFDy8YVgxB7AjpsHtMiMEX+UkxpDeUSYBEao+CPygduaGe3FEk1atoGvOVvQU2auzYEx8QF5YPBnqQaNXAiict+isohAewRdM6CS9jAy9AVesba4xD6tVmAX/BwTC8NMLEs16Cp/w9QsuJWlGlQD32B2C8ikm93ju6nPf2zqvYcw6u87JO8tKLODVi9BVGaM0lQjfn9bHzPqTu3gu2lkxmyIneTENsy6Voh3a5mPoD2YIE5UHUkddJBNOrpsVre0Hh0gjSU48iwOUDD8OltSYXVB9S6L9vxJgqjqSbpA4ox12axuXyRnPui5UBgmY3FO7MG8kg5RCzjRNNqL8ySKqFjQ4GiukwldXTb7rV5KgB1BNxVkcSGtBkEf/Ulh4FdIU2DnI0NcyUM4aI85uZpsdqExp9xxyvHxZVzvod8RHC1hbuNTeDnXnSDGY5QiKmobqPha3q2vgcuAQ/dTHQGLuykfBVEr6cKQJwlGuTJnsgMsHrjxO0mzYhgd8+WKQfE7LuCPoD4sQNJNBXWAyEpyLRsmZoUbTbFxSZ6PES+y7wEG8vY/B56x+Knwv9WhExeMpafscRFHZCVhnLZID4t2YCgUt27yzzGitCIOZhq+kK4YyOWWyl2DXCq6yUYuDGIGiAsxMkPGhUJxquEdGERZFQ52jT8HDngozseozrLimmpiuWn0CooD5taHceLsPyiRD+OE2EUkDjsZ1XOxZYczDTbWRGtTDlHVipw1neazhDM4SP/LPbzMBH93Giek0UOCKAlTQXxLSuDqWGbMISrGorYgWpCuLMAqnrFM7EiCKPFegIe8EkYSAIg7v1NqyaJ+Wum6qCrXMVNPlVa5U/cjPMMPxmakOysjuq3PpV4bqc8tcxfGjSK18ywETBGFPRUERnEJdqj4ZivX5/5tJAngqnXTaiKpRtbdUkThSjiQzseuRVUrC2x+PnsgvsxBX+O/QyTVyGpmWZwk2KwBbXoyrHz5rDyfgEeauqLhWOpamlsVRaR5S2SIB977w66UxFby5GOKhMWsRd+37Fg4lepE+vsVZcCJ3zfZw2D4CSpQAU6LkVg6bxK5Iy7Pi186DvqR5vP5d6TRaPRTem7uNrlMD8wR+a39oAKcWSTpEhkS/Sq68RO1Zw6fzBEH0DXCBslGq2zSkeZkuqUXz9E780szaRkM4uCwym2uZLI9np9QF3uepBFT+mEQr2BdA6gRM1MfksJjHN9zNbISGggk+6VEz3KDY84Ydja5LibG0GWwY1XcUsn8BJeba3X9hzj9k5jcX8GziDOArNFnEYs1CNhU7A8gnOfImhkpw1heE9FqA8k6N/EcE0wuCwFtoURSfFmxRlyo7whnAfJm9jVrf4ACUWggCaLE9Tzl+C8g9thmBMtRik3swVkIs9DM3qlCEnEQNozYwUoq68pbCouFLqbHwIyisJRQ9AUKXchxSt/HKuymEs80kZyU4Qp+cQki04wwTAMFCsSdsh4WchXs7XXLUBOdxAml8OYIYZc2NEksKFfm8w+g7afAo/AFDhjHY9cLQv3181+yspFg9aA091TW7AHiKuvrdv/Uz3Xi6hOY/W/yhMgqusg9f/dVOkk3peERJ+m+bvxOv80pOEWab5SIzAyOXRD/5jaQ7NMxfFD3xvkrOcmfzd/QgWUJiLhp/YnMXCLKneHf9nMMuLlXHrHtR9zzUQB/n8a41RuMBKGl4OKXdt7AFEtUAhUgtveCIsHqODFia6+ZirqpaOmu8Cqtth5fKL4uTIjY1q4q6qayO9/WrbSqrjhSlVxO+NPCAMCWrCqVIC7acOdyUdiTpP6yWzQPrRuOviwtll73emmZ53ClJWr5pb2ahV0Nk6J6K0ds1Z228vtslbdL75p/kX0qrFqIp0A0r60xOcplAypEs9eS3FFdflcimutWnL/hqef71IjmsQWRnKtZ+qFBNE+Nd4+WblmVDtGcN5zRkl0NXh6x4YyWfsJWj9hoRn0blkI0+421OW6Z5Y1lEM2gob7DU212qoZoXhoZAyDtPvIKiOasgbFc2en2kojml9+0mNxX76+ujmgunUblj7alOUvkBkRz+94g52G9l19HXR4xClgbY3RQKVN6A6IZNmNAYr/S2qVKiOZu34DO6uz1iyRuRySd9cUNiSt10lsQzYP1UstqK1cR1YNobvuva0iM5lV2pNyKaJor/lDJ58hRb0+vEdGcHF9hWrEfVG/CWxHJDrdnz+tg93zjzo4bEcmKwKf6D8etuI63BkRzskZPG5I2+qi0S64mRNNcHP2nOBDbD+7Z/HEPItkO/nhI2+9Xi2bqRSSQ6JEXcWHHD+4DvB8x6q4X71EX/2HL+7h/f9L9iJHh6e2HD+ivtrfn9kzfojoQI21Ovmp3e3Vhy+/XtBG3JsQodO29o7oosYPeZ7VtvaoNMdLu91wDJbbQ/u1eE8OqTsRIu/GP795+UzW2Xf99zO2PuE81I0bafh4xugXTtpATfNa/NbB+RKJdeDwjzyrLiW3LQ/ugV2f3zPUYRKLFdRyc/aFrOQpScnfB0N+fxtfHbex8HGKs3WH21z97Q0TORqXn99FLFFzX89DQ28//xodlHc5PoUcjJtoOppuvsDcbj9/e3sazXvh1nQ5uSnCr60mIr9T/AeL/ACEF6W1o6POxAAAAAElFTkSuQmCC'

In [22]:
for row in logos.itertuples():
    save_path = "img/{}.png".format(row.team_id)
    if row.logo_url == '':
        src = default_address
        urllib.request.urlretrieve(src, save_path)
    else:
        src = row.logo_url
        r = requests.get(src, stream=True)
        if r.status_code == 200:
            with open(save_path, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
#     print(src[:20])
#     print(save_path)